In [13]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
from input_data import load_data
from preprocessing import preprocess_graph, sparse_to_tuple, mask_test_edges
np.set_printoptions(threshold=np.inf)




W_gene      =  pd.read_table( "GBM_Adjacency_matrices/gene.txt", delim_whitespace=True).to_numpy()
W_methy     =  pd.read_table( "GBM_Adjacency_matrices/methy.txt", delim_whitespace=True).to_numpy()
W_mirna     =  pd.read_table( "GBM_Adjacency_matrices/mirna.txt", delim_whitespace=True).to_numpy()

gene      =  pd.read_table( "GBM/GLIO_Gene_Expression.txt", delim_whitespace=True).to_numpy().transpose()
methy     =  pd.read_table( "GBM/GLIO_Methy_Expression.txt", delim_whitespace=True).to_numpy().transpose()
mirna     =  pd.read_table( "GBM/GLIO_Mirna_Expression.txt", delim_whitespace=True).to_numpy().transpose()
truelabel =  pd.read_table( "GBM/GLIO_Survival.txt", delim_whitespace=True).to_numpy()



print(gene.shape)
print(methy.shape)
print(mirna.shape)

def get_nomralized_laplacian(a):
    b = np.diag(np.power(1.0 / a.sum(axis=1), 0.5))
    return (b @ a @ b)




#features = np.c_[mirna,methy]
# adj = W_methy
# features = np.c_[gene,mirna]

adj = W_gene
features = mirna




#pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
#norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

pos_weight = 1
norm = 1







(215, 12042)
(215, 1305)
(215, 534)


In [18]:
#Model definition Cell

adj_norm = get_nomralized_laplacian(adj)
num_features = features.shape[1]

adj_label_tensor = torch.Tensor(adj)
adj_norm_tensor  = torch.Tensor(adj_norm)
features_tensor  = torch.Tensor(features)

# from the previous cell following arrays are obtained : 
#adj_label
#adj_norm
#features
#val_edges_false
#test_edges
#test_edges_false




class Model(torch.nn.Module):

    def __init__(self,A_tilda,**kwargs):
        super(Model, self).__init__(**kwargs)
        
        
        
            

        self.W0 = torch.nn.Linear(num_features,400)
        self.W3 = torch.nn.Linear(400,num_features)
        self.A_tilda = A_tilda

        
        param_list = []
        param_list += list(self.W0.parameters())
        #param_list += list(self.W1.parameters())
        #param_list += list(self.W2.parameters())
        param_list += list(self.W3.parameters())
        
        

        self.optimizer = torch.optim.Adam(param_list,lr=0.001)
        self.recon = None
        
        
        

    


        
    def forward(self,x) : 
        self.layer_1 = torch.nn.functional.relu(torch.matmul(self.A_tilda,self.W0(x)))
        self.layer_4 = self.W3(self.layer_1)


        self.x_prim = self.layer_4
        self.embedding = self.layer_1
        self.recon_loss = ((self.x_prim-x)**2).sum(1).mean(0)
        self.loss = self.recon_loss
        return self.loss.item()
        
    def backward(self) :
        
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        
        
    def get_cosine_sim_emb(self):
        
        emb = self.embedding.detach().numpy()

        norms =  (emb*emb).sum(1).reshape(-1,1)**0.5
        normalized_emb = emb/norms

        emb_matrix = np.abs( normalized_emb@normalized_emb.transpose())
        return emb_matrix
        

    def get_euclidian(self):
        emb = self.embedding.detach().numpy()
        
        emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
        for i in range(emb.shape[0]):
            for j in range(emb.shape[0]):
                emb_matrix[i,j] = ((emb[i]-emb[j])**2).sum()
                
        return emb_matrix
        
        



In [19]:
#Training Cell 

my_model = Model(A_tilda = adj_norm_tensor)


for epoch_counter in range(20000):
    
    epoch_loss = my_model.forward(features_tensor)
    my_model.backward()
    
    if epoch_counter % 100 ==0:
        print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(epoch_loss))


print('\n',"#########################################################",'\n')






Epoch: 0001 train_loss= 534.35931
Epoch: 0101 train_loss= 320.00107
Epoch: 0201 train_loss= 194.38043
Epoch: 0301 train_loss= 124.89735
Epoch: 0401 train_loss= 82.11608
Epoch: 0501 train_loss= 55.30250
Epoch: 0601 train_loss= 38.23806
Epoch: 0701 train_loss= 27.31450
Epoch: 0801 train_loss= 20.04575
Epoch: 0901 train_loss= 15.08217
Epoch: 1001 train_loss= 11.51448
Epoch: 1101 train_loss= 8.99443
Epoch: 1201 train_loss= 7.08684
Epoch: 1301 train_loss= 5.62267
Epoch: 1401 train_loss= 4.51720
Epoch: 1501 train_loss= 3.66666
Epoch: 1601 train_loss= 3.02589
Epoch: 1701 train_loss= 2.58007
Epoch: 1801 train_loss= 2.12159
Epoch: 1901 train_loss= 1.80101
Epoch: 2001 train_loss= 1.51179
Epoch: 2101 train_loss= 1.30860
Epoch: 2201 train_loss= 1.14411
Epoch: 2301 train_loss= 0.98876
Epoch: 2401 train_loss= 0.88739
Epoch: 2501 train_loss= 0.79374
Epoch: 2601 train_loss= 0.78469
Epoch: 2701 train_loss= 0.61262
Epoch: 2801 train_loss= 0.48865
Epoch: 2901 train_loss= 0.45837
Epoch: 3001 train_loss= 0

KeyboardInterrupt: 

In [20]:
#my_model.forward(features_tensor)
emb = my_model.embedding.detach().numpy()
        
emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
for i in range(emb.shape[0]):
    for j in range(emb.shape[0]):
        #emb_matrix[i,j] =  ((emb[i]-emb[j])**2).sum()
        emb_matrix[i,j] =  np.exp(-((emb[i]-emb[j])**2).sum()/150)
                
print(emb_matrix)

[[1.00000000e+00 7.70564538e-01 6.21667222e-01 6.93221299e-01
  3.97143565e-01 4.55606587e-01 1.89558314e-03 4.53417947e-01
  4.77119060e-01 6.27479351e-01 7.62584295e-01 7.47632840e-01
  6.71450745e-01 4.61212878e-01 5.90920005e-01 6.72902344e-01
  7.63269310e-01 7.74201971e-01 6.69417617e-01 6.16908938e-01
  6.85603993e-01 1.91949342e-01 1.44722706e-02 1.22186859e-01
  1.40218869e-02 1.81506498e-01 1.39785521e-02 1.39151788e-01
  2.81004052e-01 2.36706128e-01 1.73102591e-01 3.50743642e-01
  4.58718572e-02 1.11068852e-01 4.46709806e-01 3.85890338e-03
  2.71111772e-01 3.05123156e-01 1.73281378e-01 7.94803965e-02
  1.23899509e-02 3.99097086e-02 1.95836967e-02 2.20729758e-03
  1.02007417e-01 2.20605848e-01 5.57696269e-02 3.37405201e-02
  2.91102469e-02 2.16448942e-01 7.21666639e-02 9.48174323e-02
  1.10310274e-01 3.19592798e-01 9.50632819e-02 4.41729108e-02
  2.46022282e-01 2.49160262e-02 2.45772955e-02 1.28487033e-01
  2.92711705e-01 1.19774464e-01 5.18902341e-02 6.28018120e-03
  1.0207

In [23]:
np.savetxt("gvae_fusion_results/emb_sade.txt",emb,delimiter='\t')
np.savetxt("gvae_fusion_results/res_sade.txt",emb_matrix,delimiter='\t')

In [21]:
from sklearn.cluster import KMeans
X = emb
n_clusters=3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

for i in range(n_clusters):
    indx = np.argwhere(kmeans.labels_==i).reshape(-1)
    print(truelabel[indx][:,1].mean())



626.8765432098766
536.3030303030303
573.3168316831683


In [21]:
emb_matrix = my_model.get_cosine_sim_emb()
print(emb_matrix)
np.savetxt("gvae_fusion_results/res_cosine.txt",emb_matrix,delimiter='\t')

[[9.99999762e-01 1.07528433e-01 2.69530844e-02 9.13520753e-02
  1.03215739e-01 7.80488998e-02 5.88160381e-02 2.92040147e-02
  3.22823524e-02 2.40955632e-02 2.25115180e-01 5.83793297e-02
  6.07349798e-02 7.87358359e-02 9.43368226e-02 6.90505430e-02
  2.34133713e-02 4.30251881e-02 3.36033434e-01 3.34922850e-01
  1.67553216e-01 1.98978633e-02 2.84192488e-02 2.92781666e-02
  1.31397210e-02 3.53594646e-02 2.65772641e-01 4.52555791e-02
  4.13791835e-02 2.36687094e-01 9.38652232e-02 1.55723453e-01
  2.88900912e-01 2.28089005e-01 5.03865629e-03 2.44999051e-01
  1.20611086e-01 2.92277858e-02 1.25792027e-01 4.08527851e-02
  2.19282031e-01 1.06328651e-01 9.04962942e-02 2.38926888e-01
  1.54283881e-01 2.10222602e-01 1.37868032e-01 2.08886683e-01
  2.64776915e-01 2.95316100e-01 6.46592751e-02 5.15146069e-02
  4.34365980e-02 1.80428043e-01 1.37596533e-01 1.18417107e-01
  1.36523679e-01 8.89368914e-03 1.69216335e-01 1.62840560e-02
  3.34962867e-02 1.38020709e-01 1.60055198e-02 5.96024096e-02
  1.8452